# Week 5, Class 3: Data Cleaning and Manipulation

In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('experiment_data.csv')
df['Ratio'] = df['pH_Level']/df['Weight(g)']
print(df)

  SampleID  Weight(g)  pH_Level Catalyst     Ratio
0    S-001       10.5       7.2        A  0.685714
1    S-002       12.1       7.5        B  0.619835
2    S-003        9.8       6.9        A  0.704082
3    S-004       11.3       7.1        C  0.628319
4    S-005       13.5       7.3        B  0.540741


In [5]:
# df = df.drop('Ratio', axis=1)
del df['Ratio']
print(df)

  SampleID  Weight(g)  pH_Level Catalyst
0    S-001       10.5       7.2        A
1    S-002       12.1       7.5        B
2    S-003        9.8       6.9        A
3    S-004       11.3       7.1        C
4    S-005       13.5       7.3        B


In [ ]:
# ~

## Task 1: Load the 'Experiments' sheet. How many rows and columns are there?

First, we need to load the data from the `Experiments` sheet. We can then use the .shape attribute to get the number of rows and columns.

In [6]:
# Load the data from the 'Experiments' sheet (CSV file)
experiments_df = pd.read_excel('data.xlsx', sheet_name='Experiments')

# Use the .shape attribute to get the number of rows and columns
rows, columns = experiments_df.shape
print(rows, columns)

355 10


## Task 2: Show the first 8 rows and the last 3 rows.

We can use the `.head()` and `.tail()` methods to view specific numbers of rows from the beginning and end of the DataFrame, respectively.

In [7]:
# Show the first 8 rows
first_8_rows = experiments_df.head(8)

# Show the last 3 rows
last_3_rows = experiments_df.tail(3)

print(first_8_rows)
print(last_3_rows)

     exp_id            species treatment  dose_mg  response  replicate  \
0  EXP_0000        A. thaliana    drug_B        0 -0.711957          3   
1  EXP_0001        A. thaliana    drug_A      100  4.320830          4   
2  EXP_0002       m. musculus    control       10 -0.318308          3   
3  EXP_0003   D. melanogaster     drug_A        0  0.682846          1   
4  EXP_0004   D. melanogaster     drug_B       50       NaN          3   
5  EXP_0005            E. coli    drug_B       50  0.634457          1   
6  EXP_0006       m. musculus     drug_B        0 -1.020293          3   
7  EXP_0007        A. thaliana   control        5 -0.155242          3   

         date    lab  site_id    notes  
0  2025-07-24  Lab-1      138       ok  
1  2025-07-07  Lab-1      135  T=21.3C  
2  2025-07-07  Lab-2      108     none  
3  2025-07-12  Lab-3      138  contam?  
4  2025-07-24  Lab-3      129   repeat  
5  2025-07-07  Lab-2      117       ok  
6  2025-07-19  Lab-2      124   repeat  
7  20

## Task 3: What are the dtypes? Convert 'dose_mg' to float and 'date' to datetime.

The `.dtypes` attribute shows the data type of each column. We'll use `pd.to_numeric()` to handle the `dose_mg` column, which may contain non-numeric values, and `pd.to_datetime()` for the date column.

In [8]:
# Check original dtypes
print(experiments_df.dtypes)

# Convert 'dose_mg' to float
experiments_df['dose_mg'] = pd.to_numeric(experiments_df['dose_mg'])

# Convert 'date' to datetime
experiments_df['date'] = pd.to_datetime(experiments_df['date'])

# Check new dtypes
print()
print(experiments_df.dtypes)

exp_id        object
species       object
treatment     object
dose_mg        int64
response     float64
replicate      int64
date          object
lab           object
site_id        int64
notes         object
dtype: object

exp_id               object
species              object
treatment            object
dose_mg               int64
response            float64
replicate             int64
date         datetime64[ns]
lab                  object
site_id               int64
notes                object
dtype: object


## Task 4: How many missing values are there in each column? Fill missing 'response' with the group median per 'treatment'.

We'll use `.isnull().sum()` to count missing values. For filling the missing 'response' values, we'll apply a common strategy of using the median of the corresponding treatment group, which is a more robust approach than using the overall median.

In [10]:
# Count missing values in each column
missing_values_count = experiments_df.isnull().sum()
# print(missing_values_count)

# Fill missing 'response' values with the median of their 'treatment' group
experiments_df['response'] = experiments_df['response'].fillna(
    experiments_df.groupby('treatment')['response'].transform('median')
)

print(f"Original missing values:\n{missing_values_count}\n\nMissing values after filling:\n{experiments_df.isnull().sum()}")

Original missing values:
exp_id        0
species       0
treatment     0
dose_mg       0
response     22
replicate     0
date          0
lab           0
site_id       0
notes         0
dtype: int64

Missing values after filling:
exp_id       0
species      0
treatment    0
dose_mg      0
response     0
replicate    0
date         0
lab          0
site_id      0
notes        0
dtype: int64


## Task 5: Strip whitespace and lowercase the 'species' values. Count unique species.

The `.str` accessor in Pandas allows us to apply string methods to an entire `Series`. We'll chain the `.str.strip()` and `.str.lower()` methods. Then, we can use `value_counts()` or `nunique()` to find the unique species.

In [11]:
# Strip whitespace and lowercase 'species' values
experiments_df['species'] = experiments_df['species'].str.strip().str.lower()

# Count the number of unique species
unique_species_count = experiments_df['species'].nunique()

# Show the counts for each unique species
species_counts = experiments_df['species'].value_counts()

print(f"Number of unique species: {unique_species_count}\n\nCounts of each unique species:\n{species_counts}")

Number of unique species: 5

Counts of each unique species:
species
m. musculus        76
e. coli            74
a. thaliana        72
d. melanogaster    72
c. elegans         61
Name: count, dtype: int64


## Task 6: From 'Experiments', compute mean and std of 'response' by 'treatment' and 'dose_mg'.

We'll use the powerful `.groupby()` method on two columns (`treatment` and `dose_mg`) and then use the `.agg()` method to apply both the mean and standard deviation functions to the `response` column.

In [14]:
# Group by 'treatment' and 'dose_mg', then aggregate 'response'
agg_results = experiments_df.groupby(['treatment', 'dose_mg', 'lab'])['response'].agg(['mean', 'std'])

print(agg_results)

                             mean       std
treatment dose_mg lab                      
control   0       Lab-1  0.261285  0.403055
                  Lab-2  0.000415  0.488973
                  Lab-3 -0.212387       NaN
          5       Lab-1 -0.300765  0.244102
                  Lab-2  0.193472  0.460784
                  Lab-3 -0.076791  0.545555
          10      Lab-1  0.111116  0.457885
                  Lab-2 -0.207769  0.586683
                  Lab-3  0.031207  0.504031
          20      Lab-1  0.068439  0.537659
                  Lab-2 -0.382913  0.335069
                  Lab-3 -0.188204  0.576952
          50      Lab-1  0.371146  0.461143
                  Lab-2  0.115876  0.626051
                  Lab-3  0.089432  0.505576
          100     Lab-1 -0.186395  0.255557
                  Lab-2 -0.467486  0.634878
                  Lab-3  0.129088  0.431946
drug_A    0       Lab-1  1.624033  0.540396
                  Lab-2  1.271916  0.565592
                  Lab-3  1.27196

## Task 7: In 'Sensors_TimeSeries', resample each sensor to 30-minute means. Forward-fill gaps up to 2 periods.

This task requires us to work with time-series data. We'll first load the `Sensors_TimeSeries` data and convert the `timestamp_utc` column to a proper datetime object. Then, we can set it as the index and use `.resample()` to group the data into 30-minute intervals and calculate the mean. Finally, we'll use `.ffill(limit=2)` to forward-fill any small gaps.

In [15]:
# Load the 'Sensors_TimeSeries' data
sensors_df = pd.read_excel('data.xlsx', sheet_name='Sensors_TimeSeries')
print(sensors_df.head())

# Convert timestamp_utc to datetime
sensors_df['timestamp_utc'] = pd.to_datetime(sensors_df['timestamp_utc'])

# Set the timestamp as the index and resample to 30-minute means
resampled_data = sensors_df.set_index('timestamp_utc').groupby('sensor_id').resample('30min').mean()

# Forward-fill gaps up to 2 periods
resampled_data = resampled_data.ffill(limit=2)

print()
print(resampled_data.head())

  sensor_id       timestamp_utc      value
0      S-01 2025-07-15 00:00:00  19.177910
1      S-01 2025-07-15 00:05:00        NaN
2      S-01 2025-07-15 00:10:00  19.508683
3      S-01 2025-07-15 00:15:00  19.829811
4      S-01 2025-07-15 00:20:00  19.092793

                                   value
sensor_id timestamp_utc                 
S-01      2025-07-15 00:00:00  19.420979
          2025-07-15 00:30:00  19.440383
          2025-07-15 01:00:00  19.719512
          2025-07-15 01:30:00  19.314704
          2025-07-15 02:00:00  19.982121


## Task 8: Merge 'Gene_Expression_Wide' with 'Sample_Metadata' on sample_id (columns starting with 'S'). Bring 'condition' and 'batch' alongside expression values.

We need to merge the `Gene_Expression_Wide` and `Sample_Metadata` DataFrames on a common `sample_id` column. We'll use `pd.merge()` for this and specify the columns to merge on.

In [18]:
# Load the necessary sheets
gene_expression_df = pd.read_excel('data.xlsx', sheet_name='Gene_Expression_Wide')
sample_metadata_df = pd.read_excel('data.xlsx', sheet_name='Sample_Metadata')

# Merge the dataframes on the 'sample_id' columns
# Note: In 'Gene_Expression_Wide', the sample IDs are columns, so we need to transpose it first
# or melt it to a long format. A simpler approach is to merge the metadata into a temporary
# transposed version and then join. A much cleaner way is to first melt the gene expression data.
# For simplicity, we'll use a direct merge with a re-shaped dataframe.
# Let's adjust the sample_metadata to have 'sample_id' as the index for easier joining.
sample_metadata_df.set_index('sample_id', inplace=True)
gene_expression_df.rename(columns={'gene': 'gene_id'}, inplace=True)
gene_expression_df.set_index('gene_id', inplace=True)
merged_df = gene_expression_df.transpose().merge(sample_metadata_df[['condition', 'batch']], left_index=True, right_index=True)

print(merged_df.head())

     Gene_001  Gene_002  Gene_003  Gene_004  Gene_005  Gene_006  Gene_007  \
S01  3.454046 -1.327933  0.079215 -1.989168  0.542515       NaN  1.119087   
S01  3.454046 -1.327933  0.079215 -1.989168  0.542515       NaN  1.119087   
S02  0.285124  0.533252 -1.417009  0.540610  3.271026 -1.048592 -0.466789   
S03 -0.380625 -1.345471  2.355821       NaN -1.192482 -0.752036  0.867776   
S04  1.420028       NaN -0.028942  1.389560 -0.689345  0.222132 -0.671149   

     Gene_008  Gene_009  Gene_010  ...  Gene_053  Gene_054  Gene_055  \
S01 -2.539208  1.096142  0.100143  ... -0.079846 -0.478333 -1.977279   
S01 -2.539208  1.096142  0.100143  ... -0.079846 -0.478333 -1.977279   
S02  0.797859 -0.722360 -1.495660  ... -0.060314 -0.935733  1.353836   
S03 -0.463745 -0.594727  0.116468  ...  2.996239 -1.188199  0.392062   
S04 -0.410341       NaN -0.933226  ... -1.077228 -1.651178  1.010781   

     Gene_056  Gene_057  Gene_058  Gene_059  Gene_060  condition  batch  
S01 -0.698763  0.903377  0.382

## Task 9: From 'Assays_Long', pivot to wide with one row per (sample_id, day) and columns as assay_type with mean 'value'.

The `Assays_Long` data is in a long format. We need to reshape it into a wide format where each `assay_type` is its own column. Since we need to calculate the mean value for cases with duplicate `(sample_id, day, assay_type)` combinations, `pd.pivot_table()` is the appropriate function to use.

In [ ]:
# Load the 'Assays_Long' data
assays_df = pd.read_excel('data.xlsx', sheet_name='Assays_Long')

# Pivot the data to wide format, computing the mean value
pivoted_df = assays_df.pivot_table(
    index=['sample_id', 'day'],
    columns='assay_type',
    values='value',
    aggfunc='mean'
)

print(pivoted_df.head())

## Task 10: Join 'Geo_Sites' to 'Experiments' using 'site_id'. Compute average response by region.

We will merge the `Geo_Sites` DataFrame with the cleaned `Experiments` DataFrame on the `site_id` column. After the merge, we can use `groupby()` to calculate the average response for each region.

In [ ]:
# Load the 'Geo_Sites' data
geo_sites_df = pd.read_excel('data.xlsx', sheet_name='Geo_Sites')

# Merge the two dataframes on 'site_id'
experiments_with_geo = pd.merge(experiments_df, geo_sites_df, on='site_id', how='left')

# Compute the average response per region
average_response_by_region = experiments_with_geo.groupby('region')['response'].mean()

print(average_response_by_region)

## Task 11: In 'Inventory' and 'Inventory_Updates', perform a left join on 'sku'. Compute the new stock = stock + delta (NaN -> 0).

This task combines merging and handling missing values. We'll perform a left merge from `Inventory` to `Inventory_Updates`. The `delta` column in the merged DataFrame will have `NaN` values for items without updates. We will fill these `NaN`s with 0 before calculating the `new_stock`.

In [ ]:
# Load the necessary sheets
inventory_df = pd.read_excel('data.xlsx', sheet_name='Inventory')
updates_df = pd.read_excel('data.xlsx', sheet_name='Inventory_Updates')

# Perform a left merge on 'sku'
inventory_merged = pd.merge(inventory_df, updates_df, on='sku', how='left')

# Fill NaN values in 'delta' with 0
inventory_merged['delta'] = inventory_merged['delta'].fillna(0)

# Compute the new stock
inventory_merged['new_stock'] = inventory_merged['stock'] + inventory_merged['delta']

print(inventory_merged.head())

## Task 12: Detect duplicated rows in 'Experiments' (full-row duplicates) and drop them keeping the first occurrence.

Pandas' `drop_duplicates()` method is perfect for this. It can detect and remove rows that are identical across all columns.

In [ ]:
# Count original rows
original_rows = len(experiments_df)

# Drop duplicate rows, keeping the first occurrence
df_no_dupes = experiments_df.drop_duplicates()

# Count rows after dropping duplicates
rows_after_dupes = len(df_no_dupes)

print(f"Original number of rows: {original_rows}",
      f"Number of rows after dropping duplicates: {rows_after_dupes}", 
f"Dropped {original_rows - rows_after_dupes} duplicate rows.", sep='\n')

## Task 13: Parse the 'notes' column in 'Experiments' and extract any temperature pattern like 'T=23.5C' into a new column 'temp_C'.

We'll use the `.str.extract()` method with a regular expression to find and capture the temperature value. The regular expression `r'T=(\d+\.?\d*)C'` is designed to match a temperature pattern, capturing the number part, and it also handles cases where there's no decimal point.

In [ ]:
# Extract the temperature pattern into a new column
experiments_df['temp_C'] = experiments_df['notes'].str.extract(r'T=(\d+\.?\d*)C')

# Convert the new column to a numeric type
experiments_df['temp_C'] = pd.to_numeric(experiments_df['temp_C'])

print(experiments_df[['notes', 'temp_C']].head())